# i) Modelo gaussiano

In [1]:
import os
import numpy as np
import pandas as pd
import clustering

## Datasets

In [2]:
DATA_BASE_PATH = "./data"

FAC_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fac")
FOU_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fou")
KAR_FILE = os.path.join(DATA_BASE_PATH, "mfeat-kar")

fac = np.loadtxt(FAC_FILE, dtype=int)
fou = np.loadtxt(FOU_FILE, dtype=float)
kar = np.loadtxt(KAR_FILE, dtype=float)

## Importando melhor resultado

In [3]:
best_result = clustering.import_best_result("data/melhor_resultado_todas.pickle")
partition, y_true = clustering.get_hard_patitions(best_result["membership_degree"])


## Probabilidade à priori das classes

In [4]:
Pw = np.array([len(c)/2000 for c in partition])
Pw

array([0.1175, 0.116 , 0.1355, 0.1465, 0.0405, 0.01  , 0.1305, 0.151 ,
       0.0815, 0.071 ])

## Densidades por dataset

## Prob. à posteriori por view

In [5]:
def calc_prob_posteriori(p_x_w, Pw):
    qtd_x, qtd_w = p_x_w.shape 
    p_w_x = np.empty((qtd_x, qtd_w))
    
    for k in range(qtd_x):
        for i in range(qtd_w):
            sum_all = np.dot(p_x_w[k], Pw)
            p_w_x[k,i] = (p_x_w[k,i] * Pw[i])/sum_all
       
    return p_w_x

## Regra da soma

In [6]:
def regra_soma_padrao(views, Pw):
    qtd_x = views[0].shape[0]
    qtd_w = len(Pw)
    x_sum_w = np.empty((qtd_x, qtd_w))

    for i in range(qtd_x):
        for k in range(qtd_w):
            views_sum = sum([v[i,k] for v in views])
            x_sum_w[i,k] = (1-len(views))*Pw[k] + views_sum
                    
    y_pred = x_sum_w.argmax(axis = 1) 
    return y_pred

## Estimador do scikit

In [7]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

class ClassificaforBayesiano(BaseEstimator, ClassifierMixin):
    def __init__(self, partition, Pw):
        self.partition = partition
        self.Pw = Pw
    
    def fit(self, X, y):
        X, y = check_X_y(X, y)
         
        self.classes_ = unique_labels(y)
        self._fit_gaussian_bayesian_data(X)
        self.X_ = X
        self.y_ = y
        return self
    
    def _fit_gaussian_bayesian_data(self, X):
        n, d = X.shape
        qtd_w = len(self.partition)
        self.means = np.array([X[idxs].mean(axis=0) for idxs in self.partition])
        self.var = np.array([((X[idxs]-self.means[i])**2).mean(axis=0) for i, idxs in enumerate(self.partition)])
        self.cov_matrix = [np.zeros((d,d)) for _ in range(qtd_w)]

        for i in range(qtd_w):
            np.fill_diagonal(self.cov_matrix[i], self.var[i])

        return self
    
    def _calc_gaussian_density_prob(self, xk, cls):
        d = xk.shape[0]
        coef = np.power(2*np.pi, -d/2)
        inv_cov_matrix = np.linalg.inv(self.cov_matrix[cls])
        (sign, logdet) = np.linalg.slogdet(inv_cov_matrix)
        sqrt_det_inv_cov = np.sqrt(sign*np.exp(logdet))
        diff = xk - self.means[cls]
        exp_exp = np.dot((-1/2)*np.dot(diff.T, inv_cov_matrix), diff)
        exp_func = np.exp(exp_exp)

        return coef * sqrt_det_inv_cov * exp_func

    def predict_proba(self, X):
        check_is_fitted(self)
        X = check_array(X)
        
        desity_probs = np.empty((X.shape[0], len(self.classes_)))
        for k in range(desity_probs.shape[0]):
            for j in range(len(self.classes_)):
                desity_probs[k,j] = self._calc_gaussian_density_prob(X[k], j)
        
        post_probs = calc_prob_posteriori(desity_probs, self.Pw)
        
        return post_probs
        
        

In [14]:
class RegraSomaClasificadorBayesiano(BaseEstimator, ClassifierMixin):
    def __init__(self, partition, Pw):
#         self.views = views
        self.partition = partition
        self.Pw = Pw
        self.clfs = []        
    
    def fit(self, X, y):
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y
        
        for x in X:
            clf = ClassificaforBayesiano(self.partition, Pw)
            clf.fit(x,y)
            self.clfs.append(clf)
            
        return self
    
    def predict(self, X):
        assert len(X) == len(self.clfs)
        
        check_is_fitted(self)

        post_probs = [clf.predict_proba(x) for clf, x in zip(self.clfs, X)]
        
        return self.regra_soma(post_probs, Pw=self.Pw)
    
    def predict_with_proba(self, X):
        assert len(X) == len(self.clfs)
        
        check_is_fitted(self)

        return [clf.predict_proba(x).argmax(axis=1) for clf, x in zip(self.clfs, X)]
    
    def get_params(self, deep=True):
        return {"Pw": self.Pw, "partition": self.partition}
    
    def regra_soma(self, matrizes, Pw):
        qtd_x = matrizes[0].shape[0]
        qtd_w = len(Pw)
        x_sum_w = np.empty((qtd_x, qtd_w))
        
        for i in range(qtd_x):
            for k in range(qtd_w):
                views_sum = sum([v[i,k] for v in matrizes])
                x_sum_w[i,k] = (1-len(matrizes))*Pw[k] + views_sum
                
        y_pred = x_sum_w.argmax(axis = 1) 
        return y_pred

## Validação cruzada

In [15]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


RANDOM_SEED = 42
FOLDS = 10
REPEATS = 30
REAPEATS_GAUSSIAN = 10

def get_splited_partition(idxs, y_true):
    partition = [[] for i in range(10)]
    
    for i,indice in enumerate(idxs):
        partition[y_true[indice]].append(i)
        
    return partition
    
cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)

train_precision_scores = np.empty((cv.get_n_splits(),))
train_recall_scores = np.empty((cv.get_n_splits(),))
train_f1_scores = np.empty((cv.get_n_splits(),))
train_acc_scores = np.empty((cv.get_n_splits(),))

test_precision_scores = np.empty((cv.get_n_splits(),))
test_recall_scores = np.empty((cv.get_n_splits(),))
test_f1_scores = np.empty((cv.get_n_splits(),))
test_acc_scores = np.empty((cv.get_n_splits(),))

views = [fou, kar]
views_name = ["fou", "kar"]

for v, vname in zip(views, views_name):
    for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
        print(f"{vname}/{i+1}/{cv.get_n_splits()}", end=" ")

        test_views = [v[test_index]]
        train_views = [v[train_index]]

        local_partition = get_splited_partition(train_index, y_true)
        clf = RegraSomaClasificadorBayesiano(local_partition, Pw)

        clf.fit(train_views, y_true[train_index])

        y_pred_train = clf.predict_with_proba(train_views)[0]
        y_pred_test = clf.predict_with_proba(test_views)[0]

        scores_train = precision_recall_fscore_support(y_true[train_index], 
                                                        y_pred_train, 
                                                        average="macro")

        train_precision_scores[i] = scores_train[0]
        train_recall_scores[i] = scores_train[1]
        train_f1_scores[i] = scores_train[2]
        train_acc_scores[i] = accuracy_score(y_true[train_index], y_pred_train)

        scores_test = precision_recall_fscore_support(y_true[test_index], 
                                                       y_pred_test, 
                                                       average="macro")

        test_precision_scores[i] = scores_test[0]
        test_recall_scores[i] = scores_test[1]
        test_f1_scores[i] = scores_test[2]
        test_acc_scores[i] = accuracy_score(y_true[test_index], y_pred_test)

        report = dict(
            runs = cv.get_n_splits(),
            split=i+1,
            train_accuracy = train_acc_scores[i],
            train_precision = train_precision_scores[i],
            train_recall = train_recall_scores[i],
            train_fscore = train_f1_scores[i],
            test_accuracy = test_acc_scores[i],
            test_precision = test_precision_scores[i],
            test_recall = test_recall_scores[i],
            test_fscore = test_f1_scores[i],

        )

        if os.path.exists(f"data/gaussian_300_splits_{vname}.csv"):
            pd.DataFrame([report]).to_csv(f"data/gaussian_300_splits_{vname}.csv", mode="a", header=False, decimal=",", index=False)
        else:
            pd.DataFrame([report]).to_csv(f"data/gaussian_300_splits_{vname}.csv", mode="a", header=True, decimal=",", index=False)



/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass n_splits=10, n_repeats=30, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


fou/1/300 fou/2/300 fou/3/300 fou/4/300 fou/5/300 fou/6/300 fou/7/300 fou/8/300 fou/9/300 fou/10/300 fou/11/300 fou/12/300 fou/13/300 fou/14/300 fou/15/300 fou/16/300 fou/17/300 fou/18/300 fou/19/300 fou/20/300 fou/21/300 fou/22/300 fou/23/300 fou/24/300 fou/25/300 fou/26/300 fou/27/300 fou/28/300 fou/29/300 fou/30/300 fou/31/300 fou/32/300 fou/33/300 fou/34/300 fou/35/300 fou/36/300 fou/37/300 fou/38/300 fou/39/300 fou/40/300 fou/41/300 fou/42/300 fou/43/300 fou/44/300 fou/45/300 fou/46/300 fou/47/300 fou/48/300 fou/49/300 fou/50/300 fou/51/300 fou/52/300 fou/53/300 fou/54/300 fou/55/300 fou/56/300 fou/57/300 fou/58/300 fou/59/300 fou/60/300 fou/61/300 fou/62/300 fou/63/300 fou/64/300 fou/65/300 fou/66/300 fou/67/300 fou/68/300 fou/69/300 fou/70/300 fou/71/300 fou/72/300 fou/73/300 fou/74/300 fou/75/300 fou/76/300 fou/77/300 fou/78/300 fou/79/300 fou/80/300 fou/81/300 fou/82/300 fou/83/300 fou/84/300 fou/85/300 fou/86/300 fou/87/300 fou/88/300 fou/89/300 fou/90/300 fou/91/300 fou/92/3

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


fou/276/300 fou/277/300 fou/278/300 fou/279/300 fou/280/300 fou/281/300 fou/282/300 fou/283/300 fou/284/300 fou/285/300 fou/286/300 fou/287/300 fou/288/300 fou/289/300 fou/290/300 fou/291/300 fou/292/300 fou/293/300 fou/294/300 fou/295/300 fou/296/300 fou/297/300 fou/298/300 fou/299/300 fou/300/300 kar/1/300 kar/2/300 kar/3/300 kar/4/300 kar/5/300 kar/6/300 kar/7/300 kar/8/300 kar/9/300 kar/10/300 kar/11/300 kar/12/300 kar/13/300 kar/14/300 kar/15/300 kar/16/300 kar/17/300 kar/18/300 

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

kar/49/300 kar/50/300 kar/51/300 kar/52/300 kar/53/300 kar/54/300 kar/55/300 kar/56/300 kar/57/300 kar/58/300 kar/59/300 kar/60/300 kar/61/300 kar/62/300 kar/63/300 kar/64/300 kar/65/300 kar/66/300 kar/67/300 kar/68/300 kar/69/300 kar/70/300 kar/71/300 kar/72/300 kar/73/300 kar/74/300 kar/75/300 kar/76/300 kar/77/300 kar/78/300 kar/79/300 kar/80/300 kar/81/300 kar/82/300 kar/83/300 kar/84/300 kar/85/300 kar/86/300 kar/87/300 kar/88/300 kar/89/300 kar/90/300 kar/91/300 kar/92/300 kar/93/300 kar/94/300 kar/95/300 kar/96/300 kar/97/300 kar/98/300 kar/99/300 kar/100/300 kar/101/300 kar/102/300 kar/103/300 kar/104/300 kar/105/300 kar/106/300 kar/107/300 kar/108/300 kar/109/300 kar/110/300 kar/111/300 kar/112/300 kar/113/300 kar/114/300 kar/115/300 kar/116/300 kar/117/300 kar/118/300 kar/119/300 kar/120/300 kar/121/300 kar/122/300 kar/123/300 kar/124/300 kar/125/300 kar/126/300 kar/127/300 kar/128/300 kar/129/300 kar/130/300 kar/131/300 kar/132/300 kar/133/300 kar/134/300 kar/135/300 kar/136

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


kar/214/300 kar/215/300 kar/216/300 kar/217/300 kar/218/300 kar/219/300 kar/220/300 kar/221/300 kar/222/300 kar/223/300 kar/224/300 kar/225/300 kar/226/300 kar/227/300 kar/228/300 kar/229/300 kar/230/300 kar/231/300 kar/232/300 kar/233/300 kar/234/300 kar/235/300 kar/236/300 kar/237/300 kar/238/300 kar/239/300 

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


kar/240/300 kar/241/300 kar/242/300 kar/243/300 kar/244/300 kar/245/300 kar/246/300 kar/247/300 kar/248/300 kar/249/300 kar/250/300 kar/251/300 kar/252/300 kar/253/300 kar/254/300 kar/255/300 kar/256/300 kar/257/300 kar/258/300 kar/259/300 kar/260/300 kar/261/300 kar/262/300 kar/263/300 kar/264/300 kar/265/300 kar/266/300 kar/267/300 kar/268/300 kar/269/300 kar/270/300 kar/271/300 kar/272/300 kar/273/300 kar/274/300 kar/275/300 kar/276/300 kar/277/300 kar/278/300 kar/279/300 kar/280/300 kar/281/300 kar/282/300 kar/283/300 kar/284/300 kar/285/300 kar/286/300 kar/287/300 kar/288/300 kar/289/300 kar/290/300 kar/291/300 kar/292/300 kar/293/300 kar/294/300 kar/295/300 kar/296/300 kar/297/300 kar/298/300 kar/299/300 kar/300/300 

## Treinamento com base completa

- O objetivo aqui é determinar o desempenho do classificador por classe, tendo em vista diversas métricas de classificação. Observar os resultados no arquivo **data/gaussian_classification_report.txt**

In [11]:
from sklearn.metrics import classification_report


views = [fou, kar]
views_name = ["fou", "kar"]

for v, vname in zip(views, views_name):
    clf = RegraSomaClasificadorBayesiano(partition, Pw)

    clf.fit([v], y_true)

    y_pred = clf.predict_with_proba([v])[0]

    report = classification_report(y_true, y_pred, output_dict=False, digits=4)

    acc_score_gaussian = accuracy_score(y_true, y_pred)

    with open(f"data/gaussian_classification_report_{vname}.txt", "w") as report_file:
        report_file.write(report)


## Estimativa pontual e intervalo de confiança

In [39]:
p = acc_score_gaussian
z_padrao = 1.96 # Confiança de 95%

def calc_intervalo_confiança(p, z=z_padrao, n=2000):
    diff = z*np.sqrt(p*(1-p)/n)
    return (round(p - diff, 4), round(p+diff, 4))

print("Estimativa pontual: ", p)
print("Intervalo de confiança: ", calc_intervalo_confiança(p))

Estimativa pontual:  0.7555
Intervalo de confiança:  (0.7367, 0.7743)


# ii) K-Vizinhos

## Normalizando 


In [18]:
from sklearn.preprocessing import minmax_scale

fac_norm = minmax_scale(fac)
fou_norm = minmax_scale(fou)
kar_norm = minmax_scale(kar)

## Distâncias entre os elementos

In [21]:
from sklearn.metrics.pairwise import euclidean_distances

fac_dist = euclidean_distances(fac_norm, fac_norm)
fou_dist = euclidean_distances(fou_norm, fou_norm)
kar_dist = euclidean_distances(kar_norm, kar_norm)

## Função de densidade

In [22]:
def calc_knn_density_prob(view_dists, k, Pw, y_true):
    qtd_x = view_dists.shape[0]
    qtd_w = len(Pw)
    
    p_x_w = np.empty((qtd_x, qtd_w))
    k_vizinhos = view_dists.argsort(axis=1)[:,:k+1]
    
    for j in range(qtd_x):
        w_vizinhos = y_true[k_vizinhos[j, 1:]]
        for i in range(qtd_w):
            p_x_w[j,i] = (w_vizinhos == i).sum()/k
                    
    return p_x_w
    

## Desidades por dataset

In [43]:
# fac_knn_density_probs = calc_knn_density_prob(fac_dist, 5, Pw, y_true)
# fou_knn_density_probs = calc_knn_density_prob(fou_dist, 5, Pw, y_true)
# kar_knn_density_probs = calc_knn_density_prob(kar_dist, 5, Pw, y_true)


## Prob. à posteriori por view

In [44]:
# fac_knn_posteriori_probs = calc_prob_posteriori(fac_knn_density_probs, Pw)
# fou_knn_posteriori_probs = calc_prob_posteriori(fou_knn_density_probs, Pw)
# kar_knn_posteriori_probs = calc_prob_posteriori(kar_knn_density_probs, Pw)

## Regra da soma

In [45]:
# y_pred_knn_all = regra_soma_padrao([fac_knn_posteriori_probs, 
#                                    fou_knn_posteriori_probs, 
#                                    kar_knn_posteriori_probs], Pw)

# print("Acurácia: ", accuracy_score(y_true, y_pred_knn_all))
# print("Medida-F: ", f1_score(y_true, y_pred_knn_all, average="macro"))

## Validação cruzada

In [25]:
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import os

cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)

K = 3
views = [fac, fou, kar]
views_name = ["fac", "fou", "kar"]

for v, vname in zip(views, views_name):
    
    train_precision_scores = np.empty((cv.get_n_splits(),))
    train_recall_scores = np.empty((cv.get_n_splits(),))
    train_f1_scores = np.empty((cv.get_n_splits(),))

    test_precision_scores = np.empty((cv.get_n_splits(),))
    test_recall_scores = np.empty((cv.get_n_splits(),))
    test_f1_scores = np.empty((cv.get_n_splits(),))
    
    for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
        print(f"{vname}/{i+1}/{cv.get_n_splits()}", end=" ")

        train_views = [minmax_scale(v[train_index])]
        test_views = [minmax_scale(v[test_index])]

        train_views_dists = [euclidean_distances(v,v) for v in train_views]
        test_views_dists = [euclidean_distances(v,v) for v in test_views]


        train_views_density_probs = [calc_knn_density_prob(v, K, Pw, y_true[train_index]) 
                                     for v in train_views_dists]

        train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in train_views_density_probs]

        y_pred_train = regra_soma_padrao(train_views_post_probs, Pw)


        test_views_density_probs = [calc_knn_density_prob(v, K, Pw, y_true[test_index]) 
                                     for v in test_views_dists]

        test_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in test_views_density_probs]

        y_pred_test = regra_soma_padrao(test_views_post_probs, Pw)
        
        scores_train = precision_recall_fscore_support(y_true[train_index], 
                                                       y_pred_train, 
                                                       average="macro")
      
        
        train_precision_scores[i] = scores_train[0]
        train_recall_scores[i] = scores_train[1]
        train_f1_scores[i] = scores_train[2]
        train_acc_scores[i] = accuracy_score(y_true[train_index], y_pred_train)

        scores_test = precision_recall_fscore_support(y_true[test_index], 
                                                       y_pred_test, 
                                                       average="macro")

        test_precision_scores[i] = scores_test[0]
        test_recall_scores[i] = scores_test[1]
        test_f1_scores[i] = scores_test[2]
        test_acc_scores[i] = accuracy_score(y_true[test_index], y_pred_test)
        
        report = dict(
            runs = cv.get_n_splits(),
            split=i+1,
            train_accuracy = train_acc_scores[i],
            train_precision = train_precision_scores[i],
            train_recall = train_recall_scores[i],
            train_fscore = train_f1_scores[i],
            test_accuracy = test_acc_scores[i],
            test_precision = test_precision_scores[i],
            test_recall = test_recall_scores[i],
            test_fscore = test_f1_scores[i],
        )

        if os.path.exists(f"data/knn_300_splits_{vname}.csv"):
            pd.DataFrame([report]).to_csv(f"data/knn_300_splits_{vname}.csv", mode="a", header=False, decimal=",", index=False)
        else:
            pd.DataFrame([report]).to_csv(f"data/knn_300_splits_{vname}.csv", mode="a", header=True, decimal=",", index=False)
    

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass n_splits=10, n_repeats=30, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


fac/1/300 

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


fac/2/300 

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


kar/299/300 

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


kar/300/300 

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Treinamento com melhor valor de K

- O objetivo aqui é determinar o desempenho do classificador o valor de K com o qual ovtivemos a melhor medida-f média através validação cruzada. Observar os resultados no arquivo **data/knn_classification_report.txt**
- O melhor valor de K pode ser observado no arquivo **data/knn_best_k.csv**, gerado no experimento anterior

In [47]:
from sklearn.metrics import classification_report

BEST_K = 3
#views = [fac, fou, kar]
views = [fac, fou, kar]

train_views = [minmax_scale(v) for v in views]

train_views_dists = [euclidean_distances(v,v) for v in train_views]

train_views_density_probs = [calc_knn_density_prob(v, BEST_K, Pw, y_true) 
                             for v in train_views_dists]

train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                          for v in train_views_density_probs]

y_pred = regra_soma_padrao(train_views_post_probs, Pw)
      
acc_score_knn = accuracy_score(y_true, y_pred)
    
report = classification_report(y_true, y_pred, output_dict=False, digits=4)

with open("data/knn_classification_report.txt", "w") as report_file:
    report_file.write(report)


/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Estimativa pontual e intervalo de confiança

In [48]:
p = acc_score_knn
z_padrao = 1.96 # Confiança de 95%

print("Estimativa pontual: ", p)
print("Intervalo de confiança: ", calc_intervalo_confiança(p, z_padrao))

Estimativa pontual:  0.811
Intervalo de confiança:  (0.7938, 0.8282)


# iii) Janela de Parzen

## Função de densidade

In [27]:
def parzen_density_function(view, h, partition):
    qtd_x = view.shape[0]
    qtd_w = len(partition)
    
    p_x_w = np.empty((qtd_x, qtd_w))
    dims = view.shape[1]
    
    for i in range(qtd_w):
        n = len(partition[i])
        x_view = view[partition[i],:]
        for k in range(qtd_x):
            diff = (view[k] - x_view)/h
            gaussian_kernel = np.exp(-(diff**2)/2)/np.sqrt(2*np.pi)
            prod_dims = gaussian_kernel.prod(axis=1)
            p_x_w[k,i] = prod_dims.sum()/(n*h**dims)
            
    return p_x_w

In [50]:
# fac_parzen_density_probs = parzen_density_function(fac, 2, partition)
# fou_parzen_density_probs = parzen_density_function(fou, 2, partition)
# kar_parzen_density_probs = parzen_density_function(kar, 2, partition)

## Prob. à posteriori por view

In [51]:
# fac_prazen_posteriori_probs = calc_prob_posteriori(fac_parzen_density_probs, Pw)
# fou_prazen_posteriori_probs = calc_prob_posteriori(fou_parzen_density_probs, Pw)
# kar_prazen_posteriori_probs = calc_prob_posteriori(kar_parzen_density_probs, Pw)

## Regra da soma

In [52]:
# for h in range(2, 21, 1):
#     fac_parzen_density_probs = parzen_density_function(fac, h, partition)
#     fou_parzen_density_probs = parzen_density_function(fou, h, partition)
#     kar_parzen_density_probs = parzen_density_function(kar, h, partition)

#     fac_prazen_posteriori_probs = calc_prob_posteriori(fac_parzen_density_probs, Pw)
#     fou_prazen_posteriori_probs = calc_prob_posteriori(fou_parzen_density_probs, Pw)
#     kar_prazen_posteriori_probs = calc_prob_posteriori(kar_parzen_density_probs, Pw)

#     y_pred_prazen_all = regra_soma_padrao([fac_prazen_posteriori_probs, 
#                                        fou_prazen_posteriori_probs, 
#                                        kar_prazen_posteriori_probs], Pw)

#     print(f"h: {h}")
#     print("Acurácia: ", accuracy_score(y_true, y_pred_prazen_all))
#     print("Medida-F: ", f1_score(y_true, y_pred_prazen_all, average="macro"))

## Validação cruzada

- Variamos h de 2 a 12 para fins de comparação. Não foi possível, na validação cruzada, avaliar valores de janela maiores por limitação de tempo.

In [ ]:
cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)
train_acc_scores = np.empty((cv.get_n_splits(),))
train_f1_scores = np.empty((cv.get_n_splits(),))

test_acc_scores = np.empty((cv.get_n_splits(),))
test_f1_scores = np.empty((cv.get_n_splits(),))

H = 2
views = [fac, fou, kar]
views_name = ["fac", "fou", "kar"]

y_true = y_true.astype(int)

for v, vname in zip(views, views_name):
    
    train_precision_scores = np.empty((cv.get_n_splits(),))
    train_recall_scores = np.empty((cv.get_n_splits(),))
    train_f1_scores = np.empty((cv.get_n_splits(),))
    train_acc_scores = np.empty((cv.get_n_splits(),))

    test_precision_scores = np.empty((cv.get_n_splits(),))
    test_recall_scores = np.empty((cv.get_n_splits(),))
    test_f1_scores = np.empty((cv.get_n_splits(),))
    test_acc_scores = np.empty((cv.get_n_splits(),))
    
    for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
        print(f"{vname}/{i+1}/{cv.get_n_splits()}", end=" ")

        train_views = [v[train_index]]
        test_views = [v[test_index]]

        
        y_true_split = get_splited_partition(train_index, y_true)
        

        train_views_density_probs = [parzen_density_function(v, H, y_true_split) 
                                     for v in train_views]

        train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in train_views_density_probs]

        y_pred_train = regra_soma_padrao(train_views_post_probs, Pw)


        y_true_split = get_splited_partition(test_index, y_true)
        
        test_views_density_probs = [parzen_density_function(v, H, y_true_split) 
                                     for v in test_views]

        test_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in test_views_density_probs]

        y_pred_test = regra_soma_padrao(test_views_post_probs, Pw)

        scores_train = precision_recall_fscore_support(y_true[train_index], 
                                                       y_pred_train, 
                                                       average="macro")

        train_precision_scores[i] = scores_train[0]
        train_recall_scores[i] = scores_train[1]
        train_f1_scores[i] = scores_train[2]
        train_acc_scores[i] = accuracy_score(y_true[train_index], y_pred_train)

        scores_test = precision_recall_fscore_support(y_true[test_index], 
                                                       y_pred_test, 
                                                       average="macro")

        test_precision_scores[i] = scores_test[0]
        test_recall_scores[i] = scores_test[1]
        test_f1_scores[i] = scores_test[2]
        test_acc_scores[i] = accuracy_score(y_true[test_index], y_pred_test)
        
        report = dict(
            runs = cv.get_n_splits(),
            split=i+1,
            train_accuracy = train_acc_scores[i],
            train_precision = train_precision_scores[i],
            train_recall = train_recall_scores[i],
            train_fscore = train_f1_scores[i],
            test_accuracy = test_acc_scores[i],
            test_precision = test_precision_scores[i],
            test_recall = test_recall_scores[i],
            test_fscore = test_f1_scores[i],
        )

        if os.path.exists(F"data/parzen_300_splits_{vname}.csv"):
            pd.DataFrame([report]).to_csv(f"data/parzen_300_splits_{vname}.csv", mode="a", header=False, decimal=",", index=False)
        else:
            pd.DataFrame([report]).to_csv(f"data/parzen_300_splits_{vname}.csv", mode="a", header=True, decimal=",", index=False)

/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass n_splits=10, n_repeats=30, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


fac/1/300 fac/2/300 fac/3/300 fac/4/300 fac/5/300 fac/6/300 fac/7/300 fac/8/300 fac/9/300 fac/10/300 fac/11/300 fac/12/300 fac/13/300 fac/14/300 fac/15/300 fac/16/300 fac/17/300 fac/18/300 fac/19/300 fac/20/300 fac/21/300 fac/22/300 fac/23/300 fac/24/300 fac/25/300 fac/26/300 fac/27/300 fac/28/300 fac/29/300 fac/30/300 fac/31/300 fac/32/300 fac/33/300 fac/34/300 fac/35/300 fac/36/300 fac/37/300 fac/38/300 fac/39/300 fac/40/300 fac/41/300 fac/42/300 fac/43/300 fac/44/300 fac/45/300 fac/46/300 fac/47/300 fac/48/300 fac/49/300 fac/50/300 fac/51/300 fac/52/300 fac/53/300 fac/54/300 fac/55/300 fac/56/300 fac/57/300 fac/58/300 fac/59/300 fac/60/300 fac/61/300 fac/62/300 fac/63/300 fac/64/300 fac/65/300 fac/66/300 fac/67/300 fac/68/300 fac/69/300 fac/70/300 fac/71/300 fac/72/300 fac/73/300 fac/74/300 fac/75/300 fac/76/300 fac/77/300 fac/78/300 fac/79/300 fac/80/300 fac/81/300 fac/82/300 fac/83/300 fac/84/300 fac/85/300 fac/86/300 fac/87/300 fac/88/300 fac/89/300 fac/90/300 fac/91/300 fac/92/3

## Treinamento com melhor valor de h

- O objetivo aqui é determinar o desempenho do classificador o valor de h com o qual ovtivemos a melhor medida-f média através validação cruzada. Observar os resultados no arquivo **data/parzen_classification_report_h2.txt**
- Contudo, assim como com a validação cruzada, o modelo obteve 100% em todas as medidas, para todos os valores de h e em ambas as bases de treinamento e validação.

- Valores de h a partir de 30 causaram underflow no trenamento graças à poderação exponecial feita no cálculo da função de densidade do modelo e portanto não puderam ser computadas.

In [54]:
from sklearn.metrics import classification_report

BEST_H = 2
#views = [fac, fou, kar]
views = [fac, fou, kar]

train_views_density_probs = [parzen_density_function(v, BEST_H, partition) 
                             for v in views]

train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                          for v in train_views_density_probs]

y_pred = regra_soma_padrao(train_views_post_probs, Pw)

report = classification_report(y_true, y_pred, output_dict=False, digits=4)

acc_score_parzen = accuracy_score(y_true, y_pred)

with open(f"data/parzen_classification_report_h{BEST_H}.txt", "w") as report_file:
    report_file.write(report)


## Estimativa pontual e intervalo de confiança

In [55]:
p = acc_score_parzen
z_padrao = 1.96 # Confiança de 95%

print("Estimativa pontual: ", p)
print("Intervalo de confiança: ", calc_intervalo_confiança(p, z_padrao))

Estimativa pontual:  1.0
Intervalo de confiança:  (1.0, 1.0)


## Friedman test (p-value)

Conclusão: 
- Nossa hipótese nula $H_0$ é que os resultados entre os classifcadores não tem diferença estatística significativa. Se o pvalue for maior que $0,05$ (nível de significância padrão), aceitamos $H_0$. Caso contrário, rejeitamos.

- O valor de pvalue fora inferior ao nível de significância. Logo, nossa hipótese é rejeitada pelo teste de Friedman e concluímos, portanto, que os classificadores possuem desempenho diferente. Para identificarmos quais deles são diferentes entre si, realizamos um pós-teste


In [2]:
from scipy.stats import friedmanchisquare
import pandas as pd

knn_scores = pd.read_csv("data/knn_300_splits.csv", decimal=",")
gaussian_scores = pd.read_csv("data/gaussian_300_splits.csv", decimal=",")
parzen_scores = pd.read_csv("data/parzen_300_splits.csv", decimal=",")

knn_acc = knn_scores["test_accuracy"].values
gaussian_acc = gaussian_scores["test_accuracy"].values
parzen_acc = parzen_scores["test_accuracy"].values

friedmanchisquare(knn_acc, gaussian_acc, parzen_acc)

FriedmanchisquareResult(statistic=513.6526138279932, pvalue=2.895690222219934e-112)

## Friedman test (pós-teste)

Conclusão:

- Conclui-se que todos os 3 classicadores são diferentes entre si. Os valores na tabela são os p-values calculados com o teste de Nemenyi. Como podemos observar, todos eles são menores que $0,05$.

- É importante lembrar que o modelo gaussiano padrão foi treinado apenas com duas matrizes, fato que dificulta a comparação direta de seus resultados com os outros dois modelos treinamos com todas as 3 matrizes. Essa diferença pode ter resultado numa diferença significativa nos testes de Friedman. 

In [25]:
## Dados extraídos de script em R

data = [[1.1e-13, ""], ["< 2e-16" , "< 2e-16"]]
pd.DataFrame(data, columns="gaussian knn".split(), index="knn parzen".split())

,gaussian,knn
knn,1.1e-13,
parzen,< 2e-16,< 2e-16
